In [1]:
from configs.datasets import dataset as DatasetConfig
from configs.training import train_config as TrainConfig
from configs.distillation import distillation_config as DistillationConfig
from configs.fsdp import fsdp_config as FsdpConfig

In [2]:
dataset_config = DatasetConfig()
train_config = TrainConfig()
distill_config = DistillationConfig()
fsdp_config = DistillationConfig()
train_config.model_name = "google/gemma-3-1b-it"
distill_config.model_name = "Hannibal046/xrag-7b"
train_config.batch_size_training = 1
train_config.num_workers_dataloader = 1
dataset_config.file = "data/loaders/squad-v2-sampled.py"

In [3]:
import torch
model_name = "Hannibal046/xrag-7b"
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [4]:
from transformers import AutoTokenizer
xmistral_tokenizer = AutoTokenizer.from_pretrained(model_name)

In [5]:
from data.data_utils import get_dataloader
student_train_dataloader, student_eval_dataloader = get_dataloader(dataset_config, train_config, xmistral_tokenizer, 0, distill_config)

--> Training Set Length = 1000
--> Validation Set Length = 250


In [7]:
iter1 = iter(student_train_dataloader)

In [18]:
item = next(iter1)

In [22]:
item

{'input_ids': tensor([[  733, 16289, 28793, 24316, 28747, 32001, 28725,   690,   835,  2825,
         28747, 28792, 28748, 16289, 28793,   415,  2990,   302,  9143,   403,
         16783, 23799,   356,  4117, 28705, 28740, 28787, 28725, 28705, 28740,
         28787, 28750, 28750, 28725,   304,   403, 10806,  1987,  9143,  8897,
         28723,   415,  1141,   403,  4648,   298,  9143,  3805,   356,  5353,
         28705, 28750, 28750, 28725, 28705, 28740, 28787, 28774, 28783, 28723,
           415,  2990, 28742, 28713,  4889,   403, 28705, 28740, 28750, 28725,
         28734, 28734, 28734,   297, 28705, 28740, 28787, 28774, 28734, 28725,
           304, 28705, 28750, 28770, 28725, 28734, 28734, 28734,   297, 28705,
         28740, 28783, 28734, 28734, 28723,   415,  2990, 28742, 28713,  4889,
           403, 28705, 28750, 28781, 28725, 28734, 28734, 28734,   297, 28705,
         28740, 28783, 28740, 28734, 28725,   304, 28705, 28770, 28770, 28725,
         28734, 28734, 28734,   297, 2

In [19]:
item["input_ids"]

tensor([[  733, 16289, 28793, 24316, 28747, 32001, 28725,   690,   835,  2825,
         28747, 28792, 28748, 16289, 28793,   415,  2990,   302,  9143,   403,
         16783, 23799,   356,  4117, 28705, 28740, 28787, 28725, 28705, 28740,
         28787, 28750, 28750, 28725,   304,   403, 10806,  1987,  9143,  8897,
         28723,   415,  1141,   403,  4648,   298,  9143,  3805,   356,  5353,
         28705, 28750, 28750, 28725, 28705, 28740, 28787, 28774, 28783, 28723,
           415,  2990, 28742, 28713,  4889,   403, 28705, 28740, 28750, 28725,
         28734, 28734, 28734,   297, 28705, 28740, 28787, 28774, 28734, 28725,
           304, 28705, 28750, 28770, 28725, 28734, 28734, 28734,   297, 28705,
         28740, 28783, 28734, 28734, 28723,   415,  2990, 28742, 28713,  4889,
           403, 28705, 28750, 28781, 28725, 28734, 28734, 28734,   297, 28705,
         28740, 28783, 28740, 28734, 28725,   304, 28705, 28770, 28770, 28725,
         28734, 28734, 28734,   297, 28705, 28740, 2

In [21]:
xmistral_tokenizer.decode(item["input_ids"][0])

"[INST] Background:<xRAG>, which also means:[/INST] The city of Boston was officially incorporated on March 17, 1722, and was originally called Boston Town. The name was changed to Boston City on February 22, 1798. The city's population was 12,000 in 1790, and 23,000 in 1800. The city's population was 24,000 in 1810, and 33,000 in 1820. The city's population was 40,000 in 1830, and 54,000 in 1840. The city's population was 68,000 in 1850, and 115,000 in 1860. The city's population was 178,000"

In [13]:
from transformers import AutoConfig
from models.modeling_xmistral import XMistralForCausalLM

config = AutoConfig.from_pretrained(model_name)
MODEL_CLASS = eval(config.architectures[0])
xmistral = MODEL_CLASS.from_pretrained(
    model_name,
    torch_dtype = torch.bfloat16,
    low_cpu_mem_usage = True,
).to(device)

/home/brimmann/works/llm-recipes-2/models/checkpoint_handler.py:7: DeprecationWarning: `torch.distributed._shard.checkpoint` will be deprecated, use `torch.distributed.checkpoint` instead
  import torch.distributed._shard.checkpoint as dist_cp
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
generated_output = xmistral.generate(
        input_ids = input_ids,
        do_sample=False,
        max_new_tokens=20,
        pad_token_id=llm_tokenizer.pad_token_id,
        retrieval_embeds = relevant_embedding.unsqueeze(0),
    )